In [ ]:
import requests
from bs4 import BeautifulSoup
import http.cookiejar
import mechanize
import urllib

In [ ]:
class SessionGoogle:
    def __init__(self, url_login, url_auth, login, pwd):
        self.ses = requests.session()
        login_html = self.ses.get(url_login)
        soup_login = BeautifulSoup(login_html.content).find('form').find_all('input')
        my_dict = {}
        for u in soup_login:
            if u.has_attr('value'):
                my_dict[u['name']] = u['value']
        # override the inputs without login and pwd:
        my_dict['Email'] = login
        my_dict['Passwd'] = pwd
        self.ses.post(url_auth, data=my_dict)

    def get(self, URL):
        return self.ses.get(URL).text

In [ ]:
url_login = "https://www.chicagocityscape.com/account.php"
url_auth = "https://www.chicagocityscape.com/account.php/account.php?provider=google&src="
session = SessionGoogle(url_login, url_auth, "reclaimfairelections@gmail.com", "rDxPK7p6W@5f")
base = session.get("https://www.chicagocityscape.com/ordinances.php#/?type=zoning&search_term=O2019-331")
soup = BeautifulSoup(base, 'html.parser')

In [ ]:
for x in soup.find_all('a'):
    print(x)

In [ ]:
with requests.Session() as s:

    p = s.post("https://www.chicagocityscape.com/account.php?provider=google&src=", 
               data={'action': 'login',
        "email": 'reclaimfairelections@gmail.com',
        "password": 'rDxPK7p6W@5f'})
#     s.get('href')
    print(p.text)
#     base_page = s.get('https://www.chicagocityscape.com/ordinances.php?ordinance=6588&key=WwpHU6SLdp')
    base_page = s.get('https://www.chicagocityscape.com')

    soup = bs4.BeautifulSoup(base_page.content, 'html.parser')


In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import csv
import time

In [5]:
# Replace this with whatever topic page you'd like to scrape
ord_search_page = 'https://www.chicagocityscape.com/ordinances.php#/?type=zoning&search_term=O2019-331'

# Put your email and password in these variables, make sure to have the quotation marks around them
email = "reclaimfairelections@gmail.com"
pswd = "rDxPK7p6W@5f"


In [6]:
# Initialize webdriver
driver = webdriver.Firefox()
driver.get("https://www.chicagocityscape.com")
wait = WebDriverWait(driver, 30)

In [7]:
# Find sign-in by Google button and click it
elem = driver.find_element_by_partial_link_text('Sign in')
elem.click()
time.sleep(2)
goo = driver.find_element_by_partial_link_text('Google')
goo.click()
time.sleep(2)

In [8]:
# Enter Email address and submit
email_input = driver.find_element_by_id("identifierId")
email_input.send_keys(email)
email_submit = driver.find_element_by_id("identifierNext").click()

In [9]:
wait.until(EC.presence_of_element_located((By.ID, "password")))

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="88b8dc6a-3465-574e-a02d-b2f38e9a5312", element="04499c1a-2274-b249-8cde-72ad4c47a67f")>

In [10]:
# Enter Password and submit
pw_input = driver.find_element_by_name("password")
pw_input.send_keys(pswd)

In [12]:
pw_submit = driver.find_element_by_id("passwordNext").click()
time.sleep(10)

In [21]:
ord_page = 'https://www.chicagocityscape.com/ordinances.php#/?type=zoning&search_term=O2019-331'
driver.get(ord_page)